## Location in map with social media information
#### Author: Ernie Sumoso


In [2]:
import pandas as pd

In [10]:
df = pd.read_csv('data/tweets.csv')
df.isnull().mean()

tweet_id                        0.000000
airline_sentiment               0.000000
airline_sentiment_confidence    0.000000
negativereason                  0.373087
negativereason_confidence       0.281284
airline                         0.000000
airline_sentiment_gold          0.997268
name                            0.000000
negativereason_gold             0.997814
retweet_count                   0.000000
text                            0.000000
tweet_coord                     0.930396
tweet_created                   0.000000
tweet_location                  0.323292
user_timezone                   0.329235
dtype: float64

In [23]:
df.tail()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [28]:
df['tweet_coord'].value_counts()

tweet_coord
[0.0, 0.0]                      164
[40.64656067, -73.78334045]       6
[32.91792297, -97.00367737]       3
[40.64646912, -73.79133606]       3
[35.22643463, -80.93879965]       2
                               ... 
[40.69429232, -74.17208436]       1
[37.61833841, -122.38389799]      1
[37.61859126, -122.38385699]      1
[45.58931882, -122.5959928]       1
[40.64946781, -73.76624703]       1
Name: count, Length: 832, dtype: int64

In [45]:
df['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [59]:
colors = {'pos':'#00FF00', 'neu':'#0000FF', 'neg':'#FF0000'}
radius_factor = 10000
radius_capp = 50000

In [75]:
import folium as f
import numpy as np
import ast

def get_radius_values_dict(df):
    radius_values = {}
    for index, row in df.iterrows():
        if row['tweet_coord'] is np.nan:
            continue

        coords = row['tweet_coord']
        sent_score = row['airline_sentiment_confidence']
        sentiment = row['airline_sentiment'][:3]
        key = sentiment + coords
        radius_values[key] = radius_values.get(key, 0) + sent_score
    return radius_values


def convert_string_coords_into_list(coords_str):
    return ast.literal_eval(coords_str)


def draw_circles(fmap, radius_values):
    for key, radius in radius_values.items():

        sent = key[:3]
        coords = key[3:]

        coords_list = convert_string_coords_into_list(coords)

        final_radius = max(radius_capp, radius_factor * radius)
        color = colors[sent]

        f.Circle(radius=final_radius,
                  location = coords_list,
                  color=color,
                  weight=2,
                  fill=True).add_to(fmap)

fmap = f.Map([0,0], zoom_start=2.4)
get_radius_values_dict(df)
draw_circles(fmap, radius_values)
fmap